## Making a Flower Classifier with VGG16
#### Loading the VGG16 Model

In [2]:
from keras.applications import VGG16
# VGG16 was designed to work on 224x224 pixel input images sizes

img_rows = 224
img_cols = 224

#Load the VGG16 model
vgg16 = VGG16(weights =  'imagenet',
              include_top = False, #we are not adding top of the file as they are pretrained
             input_shape = (img_rows,img_cols,3))

58892288/58889256 [==============================] - 8s 0us/step


## Inspecting each layer

In [12]:
## Let's print our layers
for (i,layer) in enumerate(vgg16.layers):
     print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer True
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


## Let's Freeze all the top layer except 4

In [25]:
from keras.applications import VGG16

#Vgg16 was designed to work on 244x244 pixel input image sizes
img_rows= 224
img_cols = 224

#Re-load the vgg16 model without the top or fc layers
vgg16 = VGG16(weights='imagenet',
             include_top=False,
             input_shape = (img_rows,img_cols,3))

# include_top=True includes extra 4 layer of 
# 19 Flatten False
# 20 Dense False
# 21 Dense False
# 22 Dense False

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in vgg16.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(vgg16.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)
    
#Now all the layers have been set as false

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


## Making a function that returns our FC Head

In [46]:
def addTopModel(bottom_model,num_classes,D=256): #Here D = dense layer size
    """ creates the top or head of the model that will be 
    placed ontop of the bottom layer"""
    top_model = bottom_model.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(D,activation='relu')(top_model) 
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

#Note this different way of creating model. It has all the layers 

## Let's add our FC Head back onto VGG

In [49]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 17 # 17 types of flowers

FC_HEAD = addTopModel(vgg16,num_classes)

model = Model(inputs=vgg16.input,outputs=FC_HEAD)

print(model.summary())


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [58]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = './flowers/train'
validation_data_dir = './flowers/validation'

#performing data augmentation
train_datagen = ImageDataGenerator(
rescale = 1./255, #normalising
rotation_range = 20,
width_shift_range = 0.2,
height_shift_range = 0.2,
horizontal_flip=True,
fill_mode = 'nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

In [62]:
train_batchsize = 32
val_batchsize = 32

#Configuring 
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size = (img_rows,img_cols),
                                                    batch_size = train_batchsize,
                                                   class_mode = 'categorical')

validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                             target_size = (img_rows,img_cols),
                                                             batch_size = val_batchsize,
                                                             class_mode = 'categorical',
                                                             shuffle = False)

Found 1190 images belonging to 17 classes.
Found 170 images belonging to 17 classes.


15


## Training our top layer

In [71]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("./FlowerClassifier.h5",
                            monitor='val_loss',
                            mode = 'min',
                            save_best_only = True,
                            verbose = 1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

#WE put our call back into call back list
callbacks = [earlystop,checkpoint]

#compiling the model
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])


nb_train_samples = len(train_generator)
nb_validation_samples = 170
epochs = 25
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples//batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples//batch_size)

model.save("./FlowerClassifier_savedModel.h5")

Epoch 1/25
74/74 [==============================] - 25s 324ms/step - loss: 2.1424 - accuracy: 0.3921 - val_loss: 1.0243 - val_accuracy: 0.7588

Epoch 00001: val_loss improved from inf to 1.02433, saving model to .\FlowerClassifier.h5


In [95]:
# ! git filter-branch --tree-filter 'rm -rf FlowerClassifier.h5 ' HEAD
# ! git stash save
# ! git push origin main --force
! git add FlowerClassifier(VGG+imgnet).ipynb
! git commit -m "13-05-2021/23:30"
# ! git pull origin main
! git push -f origin main

fatal: pathspec 'FlowerClassifier(VGG+imgnet).ipynb' did not match any files


On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../../.virtual_documents/
	../../Batch_Normalization_LeNet_Alexnet/.ipynb_checkpoints/
	../../Batch_Normalization_LeNet_Alexnet/Fashion_classifier.h5
	../../Batch_Normalization_LeNet_Alexnet/fashion_mnist/
	../../CallBacks+Fruit_classifier/.ipynb_checkpoints/
	../../CallBacks+Fruit_classifier/Fruits-360/
	../../CallBacks+Fruit_classifier/Untitled.ipynb
	../../CallBacks+Fruit_classifier/fruit_classifier.pickle1
	../../CallBacks+Fruit_classifier/fruits_fresh_cnn1.h5
	../../Confusion_Matrix/.ipynb_checkpoints/
	../../Data_Augmentation(DogxCat)/.ipynb_checkpoints/
	../../Data_Augmentation(DogxCat)/.virtual_documents/
	../../Data_Augmentation(DogxCat)/cats_vs_dogs_test_data.npz
	../../Data_Augmentation(DogxCat)/cats_vs_dogs_test_labels.npz
	../../Data_Augmentation(DogxCat)/cats_vs_dogs_training_data.npz
	../../Data_Augmentation(DogxCat)/cats_vs_dog

Everything up-to-date


In [90]:
!git filter-branch -f --index-filter 'git rm --cached --ignore-unmatch fixtures/11_user_answer.json'

	 rewrites.  Hit Ctrl-C before proceeding to abort, then use an
	 alternative filtering tool such as 'git filter-repo'
	 (https://github.com/newren/git-filter-repo/) instead.  See the
	 filter-branch manual page for more details; to squelch this warning,
	 set FILTER_BRANCH_SQUELCH_WARNING=1.
Proceeding with filter-branch...



fatal: Cannot open 'FineTuning_TransferLearning(Monkey+flower_classifier)/Flower_Classifier/nul': No such file or directory
You need to run this command from the toplevel of the working tree.


In [ ]:
FlowerClassifier_savedModel.h5 FlowerClassifier.h5